In [1]:
import pandas as pd
import numpy as np
import torch
import copy
from sklearn import preprocessing

df = pd.read_csv('market.csv')
df

,Date,TS(10Y3M),TS(5Y3M),Infla,IndPro,M1 money,M2 money,FedFund,Unemp,S&PClose,S&PVol,Clo3MA,Clo12MA,MarketStatus
0,1962/1/1,1.37,1.26,0.001000,25.3943,145.2,337.5,1.50,5.8,68.839996,3840000,-0.024515,0.023770,1
1,1962/2/1,1.29,1.06,0.002330,25.8097,145.7,340.1,2.75,5.5,69.959999,3030000,-0.002234,0.032087,1
2,1962/3/1,1.11,0.86,0.001993,25.9482,146.0,343.1,2.75,5.6,69.550003,2950000,0.001440,0.020406,1
3,1962/4/1,1.12,0.90,0.001326,26.0035,146.4,345.5,2.50,5.6,65.239998,4150000,-0.044103,-0.042747,1
4,1962/5/1,1.20,0.96,0.000993,25.9758,146.8,347.5,2.75,5.5,59.630001,10710000,-0.079879,-0.117584,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2020/8/1,0.61,0.17,0.003703,102.6588,5391.2,18404.0,0.09,8.4,3500.310059,4342290000,0.063739,0.135885,1
704,2020/9/1,0.59,0.18,0.002033,102.5968,5502.5,18648.3,0.09,7.8,3363.000000,4722530000,-0.004483,0.080045,1
705,2020/10/1,0.79,0.29,0.000446,103.6058,5580.4,18812.0,0.09,6.9,3269.959961,4840450000,-0.031914,0.043673,1
706,2020/11/1,0.76,0.28,0.001890,104.0882,6047.6,19086.0,0.09,6.7,3621.629883,6291400000,0.059515,0.141325,1


In [5]:
import torch.optim as optim
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        self.learning_rate = 0.01
        self.matching_accept = False
    
    def forward(self, x, y):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        loss = torch.nn.functional.mse_loss(y_pred, y)
        return y_pred, loss
    
    def backward_Adam(self, loss, weight_decay=0):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer.zero_grad()
        

In [6]:
# Set up an acceptable SLFN with four hidden nodes
D_in, H, D_out = 12, 4, 1
model = TwoLayerNet(D_in, H, D_out)

print(model)
print(model.state_dict())


TwoLayerNet(
  (linear1): Linear(in_features=12, out_features=4, bias=True)
  (linear2): Linear(in_features=4, out_features=1, bias=True)
)
OrderedDict([('linear1.weight', tensor([[ 0.2723, -0.1925,  0.2222,  0.2130,  0.1865,  0.0548,  0.1574, -0.0492,
         -0.1201, -0.2204, -0.1330, -0.1993],
        [ 0.0138, -0.1791,  0.1294, -0.0224, -0.2030, -0.0647,  0.0367, -0.1841,
          0.0697, -0.2789,  0.2628,  0.1949],
        [ 0.0569,  0.1499,  0.0087,  0.1372, -0.1612,  0.2681, -0.0184,  0.0251,
         -0.1185,  0.0835, -0.1842, -0.1785],
        [ 0.2313,  0.2058,  0.0942, -0.1411,  0.1421, -0.0322, -0.0474,  0.2260,
          0.0686,  0.0778,  0.1662,  0.0238]])), ('linear1.bias', tensor([ 0.0304, -0.2251, -0.1314, -0.1876])), ('linear2.weight', tensor([[-0.2321,  0.1389, -0.4302, -0.4130]])), ('linear2.bias', tensor([-0.0926]))])


In [7]:
### MATCHING MODULE
## Helps tune weights to obtain an acceptable net.

def matching(x, y, model):
    #class1 = 1
    #class2 = 0
    
    for t in range(100):
        # forward
        y_pred, loss = model.forward(x, y)
        
        # learning goal
        alpha = y_pred[torch.where(y == class1)].min()
        beta =  y_pred[torch.where(y == class2)].max()
        
        if torch.gt(alpha, beta):
            model.matching_accept = True
            print("MATCHING MODULE finished: 執行了"+str(t+1)+"次→ Acceptable net.")
            return model
        
        else:
            # backward
            model.backward_Adam(loss)
            
            if t == 99:
                model.matching_accept = False
                print("MATCHING MODULE finished: 執行了"+str(t+1)+"次→ Unacceptable net.")
                return model

In [8]:
### REGULARIZING MODULE
## Helps further regularize weights while maintaining  an acceptable network.
## Hyperparameters:
## 100
## Optimizer(Adam)
## Learning rate(*1.2 or *0.7)

def regularizing(x, y, model):
    
    y_pred, loss = model(x, y)
    episilon_lr = 0.0001
    model.learning_rate = 0.001
 
    for t in range(100):
        # store weight
        pre_model = copy.deepcopy(model)
        
        while True:
            pre_loss = loss
            
            # backward operation to obtain w'
            model.backward_Adam(loss, weight_decay=0.1)
            
            # forward operation
            y_pred, loss = model(x, y)
            
            # learning goal
            alpha = y_pred[torch.where(y == 1)].min()
            beta =  y_pred[torch.where(y == 0)].max()
            
            if loss <= pre_loss:
                if torch.gt(alpha, beta):
                    model.learning_rate *= 1.2
                    break
                    
                else:
                    # restore weight
                    model = pre_model
                    print('REGULARIZING MODULE finished: 執行了'+str(t+1)+'次→ An acceptable net that has a preference on weights.')
                    return model
                
            else:   
                if model.learning_rate > episilon_lr:
                    # restore weight
                    model = pre_model
                    model.learning_rate *= 0.7
                           
                else:
                    # restore weight
                    model = pre_model
                    print('REGULARIZING MODULE finished: 執行了'+str(t+1)+'次→ An acceptable net that has a preference on weights.')
                    return model
                
        if t >= 99:
            print('REGULARIZING MODULE finished: 執行了'+str(t+1)+'次→ An acceptable net that has a preference on weights.')
            return model
            
            

In [9]:
### REORGANIZING MODULE
## Helps identify and remove the potentially irrelevant hidden node.
## Hyperparameters:
## 100
## optimizer(Adam, one with weight_decay)


def reorganizing(x, y, model):
    k = 1
    # current number of hidden nodes
    p = model.linear1.weight.shape[0]
    
    #alpha = y_pred[torch.where(y == 1)].min()
    #beta =  y_pred[torch.where(y == 0)].max()
    
    
    while True:

        if k > p:
            print('REORGANIZING MODULE finished: Acceptable net with the number of hidden nodes=', p)
            return model
        
        else:
            # 呼叫 regularizing module
            model = regularizing(x, y, model)
            pre_model = copy.deepcopy(model)
            
            
            model.linear1.weight = torch.nn.Parameter(torch.cat([model.linear1.weight[:k-1],model.linear1.weight[k:]]))
            model.linear1.bias = torch.nn.Parameter(torch.cat([model.linear1.bias[:k-1], model.linear1.bias[k:]]))
            model.linear2.weight = torch.nn.Parameter(torch.cat([model.linear2.weight[:,:k-1], model.linear2.weight[:,k:]],1))
            
            # 呼叫 matching module
            model = matching(x, y, model)
            
            
            if model.matching_accept == True:
                print('REORGANIZING MODULE drop the hidden node: %d/%d' %(k, p))
                p = p-1
            
            else:
                model = pre_model
                print('REORGANIZING MODULE cannot drop the hidden node: %d/%d' %(k, p))
                k = k+1
                
                
    

In [10]:
### CRAMMING MODULE 
def cramming(x, y, model):
    
    y_pred, loss = model(x, y)
    
    sub_data = (x[:-1]-x[-1])
    sub_data = torch.transpose(sub_data,0,1)
    unaccept_data = torch.transpose(x[-1].resize(1,12),0,1)
    
    while True:
        # find m-vector: gamma
        gamma = torch.rand(1,12)
        matmul_subvalue = torch.matmul(gamma, sub_data)
        if 0 not in matmul_subvalue:
            break
    
    while True:
        # find a small number: zeta
        zeta = torch.rand(1) 
        z_value = (zeta+matmul_subvalue)*(zeta-matmul_subvalue)
        if False not in torch.lt(z_value,0):
            break
    
            
    # add three hidden nodes
    # set the weight of input layer to hidden layer
    wih0 = gamma
    wih1 = gamma
    wih2 = gamma
    wih_new = torch.cat([wih0,wih1,wih2],0)
    
    # set the bias of hidden nodes
    matmul_unvalue = torch.matmul(gamma, unaccept_data)
    bh0 = zeta-matmul_unvalue
    bh1 = -matmul_unvalue
    bh2 = -zeta-matmul_unvalue
    bh_new = torch.cat([bh0,bh1,bh2],1)[0]                   
    
    # set the weight of hidden layer to output layer
    gap1 = y_pred[torch.where(y[:-1] == 1)]-y_pred[-1]
    gap2 = y_pred[torch.where(y[:-1] == 0)]-y_pred[-1]
    
    if y[-1] == 1 :
        who0 = min(gap1)/zeta
        who1 = (-2*min(gap1))/zeta
        who2 = min(gap1)/zeta
        
    else:
        who0 = max(gap2)/zeta
        who1 = (-2*max(gap2))/zeta
        who2 = max(gap2)/zeta
    
    wo_new = torch.cat([who0,who1,who2],0)
        
    #model.hidden_node +=3    
    # adjust neural network(add new hidden nodes)
    model.linear1.weight = torch.nn.Parameter(torch.cat([model.linear1.weight, wih_new]))
    model.linear1.bias = torch.nn.Parameter(torch.cat([model.linear1.bias, bh_new]))
    model.linear2.weight = torch.nn.Parameter(torch.cat([model.linear2.weight[0], wo_new]).unsqueeze(0))
    
    #print(model.state_dict())
    print('CRAMMING MODULE finished: The number of hidden nodes=', model.linear1.weight.shape[0])
    return model

In [11]:
#### 主程式
### INITIALIZING MODULE
## Use the random method to set up an acceptable SLFN with one hidden node 
## through the first two training data {(x1, y1), (x2, y2)}

# Find the first index of class1 and class2
class1 = 1
class2 = 0
first_index_of_class1 = df.index[df['MarketStatus']==class1][0]
first_index_of_class2 = df.index[df['MarketStatus']==class2][0]

# Create x data and y data
x_data = df.iloc[:,1:-1].values
y_data = df.iloc[:,-1].values

# Transform x data to 0-1 range
scaler = preprocessing.MinMaxScaler()
x_data = scaler.fit_transform(x_data)

# Change the data type to tensor
x_data = torch.FloatTensor(x_data)
y_data = torch.FloatTensor(y_data).unsqueeze(1)

# The first two training data
x_ini_data = x_data[[first_index_of_class1, first_index_of_class2]]
y_ini_data = y_data[[first_index_of_class1, first_index_of_class2]]

# Remain data
x_data = torch.cat([x_data[1:first_index_of_class2], x_data[first_index_of_class2+1:]])
y_data = torch.cat([y_data[1:first_index_of_class2], y_data[first_index_of_class2+1:]])

###################################################################################################################

### SELECTING MODULE
## Pick up the first n training data {(xc, yc): c = 1, 2, …, n} and I(n) = {1, 2, …, n}.
x_current = x_ini_data
y_current = y_ini_data

num_train = 500

#for i in range(len(x_data)):
for i in range(num_train-2):   
    print('Stage: %d' %(i+3))
    print(model.state_dict())
    
    x_current = torch.cat([x_current, x_data[i].unsqueeze(0)])
    y_current = torch.cat([y_current, y_data[i].unsqueeze(0)])
    
    y_pred, loss = model(x_current, y_current)

    ### SET THE LEARNING GOAL:SEC
    # define alpha
    if class1 in y_current:
        alpha = y_pred[torch.where(y_current == class1)].min()
    else:
        alpha = 1e6
        
    # define beta
    if class2 in y_current:
        beta = y_pred[torch.where(y_current == class2)].max()
    else:
        beta = -1e6
     
    print('alpha=',alpha)
    print('beta= ',beta)
    print('\n')
    
    # check whether the new data meets the leaning goal:SeC
    if torch.gt(alpha, beta):
        #進入reorganizing module
        model = reorganizing(x_current, y_current, model)
    
    else:
        # save current weight
        pre_model = copy.deepcopy(model)
        
        #進入matching module
        model = matching(x_current, y_current, model)
        
        if model.matching_accept == True:
            #進入reorganizing module
            model = reorganizing(x_current, y_current, model)
            
        else:
            #restore weight
            model = pre_model
            #進入cramming module
            model = cramming(x_current, y_current, model)
            #進入reorganizing module
            model = reorganizing(x_current, y_current, model)
    
    print(model.state_dict())
    print('------------------------------------------------------------------------------------------------------------------------')


Stage: 3
OrderedDict([('linear1.weight', tensor([[ 0.2723, -0.1925,  0.2222,  0.2130,  0.1865,  0.0548,  0.1574, -0.0492,
         -0.1201, -0.2204, -0.1330, -0.1993],
        [ 0.0138, -0.1791,  0.1294, -0.0224, -0.2030, -0.0647,  0.0367, -0.1841,
          0.0697, -0.2789,  0.2628,  0.1949],
        [ 0.0569,  0.1499,  0.0087,  0.1372, -0.1612,  0.2681, -0.0184,  0.0251,
         -0.1185,  0.0835, -0.1842, -0.1785],
        [ 0.2313,  0.2058,  0.0942, -0.1411,  0.1421, -0.0322, -0.0474,  0.2260,
          0.0686,  0.0778,  0.1662,  0.0238]])), ('linear1.bias', tensor([ 0.0304, -0.2251, -0.1314, -0.1876])), ('linear2.weight', tensor([[-0.2321,  0.1389, -0.4302, -0.4130]])), ('linear2.bias', tensor([-0.0926]))])
alpha= tensor(-0.1909, grad_fn=<MinBackward1>)
beta=  tensor(-0.1497, grad_fn=<MaxBackward1>)


MATCHING MODULE finished: 執行了32次→ Acceptable net.
REGULARIZING MODULE finished: 執行了100次→ An acceptable net that has a preference on weights.
MATCHING MODULE finished: 執行了1次→ Acceptab

C:\Users\user\Anaconda3\lib\site-packages\torch\tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


REGULARIZING MODULE finished: 執行了15次→ An acceptable net that has a preference on weights.
MATCHING MODULE finished: 執行了100次→ Unacceptable net.
REORGANIZING MODULE cannot drop the hidden node: 1/4
REGULARIZING MODULE finished: 執行了15次→ An acceptable net that has a preference on weights.
MATCHING MODULE finished: 執行了100次→ Unacceptable net.
REORGANIZING MODULE cannot drop the hidden node: 2/4
REGULARIZING MODULE finished: 執行了15次→ An acceptable net that has a preference on weights.
MATCHING MODULE finished: 執行了100次→ Unacceptable net.
REORGANIZING MODULE cannot drop the hidden node: 3/4
REGULARIZING MODULE finished: 執行了15次→ An acceptable net that has a preference on weights.
MATCHING MODULE finished: 執行了100次→ Unacceptable net.
REORGANIZING MODULE cannot drop the hidden node: 4/4
REORGANIZING MODULE finished: Acceptable net with the number of hidden nodes= 4
OrderedDict([('linear1.weight', tensor([[ 0.0341, -0.3883, -0.7489, -1.4182, -1.6198, -1.0689, -0.2131,  2.7523,
         -0.7608, -4.14

KeyboardInterrupt: 